In [111]:
import ipyparallel
import os
rc = ipyparallel.Client(profile="mpi")
view = rc[:]
view.apply(os.chdir, os.getcwd())
print view

<DirectView [0, 1, 2, 3,...]>


In [112]:
%%px
from mpi4py import MPI
mpi = MPI.COMM_WORLD
bcast = mpi.bcast
barrier = mpi.barrier
rank = mpi.rank
print "MPI rank: %i/%i" % (mpi.rank,mpi.size)

[stdout:0] MPI rank: 5/6
[stdout:1] MPI rank: 4/6
[stdout:2] MPI rank: 0/6
[stdout:3] MPI rank: 1/6
[stdout:4] MPI rank: 3/6
[stdout:5] MPI rank: 2/6


In [113]:
#!pwd

In [207]:
%%px --block

#Parameters
dt=0.001

T=10
Tank_he=0.02
Caisson_he=0.02
Scales=[21, 25]

#Visualization
Triangles=True
Air=False
import sys

sys.path.append('/home/user/JorgeMijares/air-water-vv/2d/Interactive/Temp_Flume_GUI')

import proteus
## Required imports
from proteus.iproteus import * 
from proteus import default_n, default_s, default_so, Comm, Context
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
#import Context

import tempFlume as plant
reload(plant)
import tempFlume_so as plant_so
reload(plant_so)

import pandas as pd
from IPython.display import clear_output

xl = pd.ExcelFile("DoE_TempFlume.xlsx")
df1 = xl.parse(0)
df2 = xl.parse(1)
df3 = xl.parse(2)
df4 = xl.parse(3, header=0, index_col=0)
df5 = xl.parse(4, header=0, index_col=0)

scale=21
index=0

clear_output()
reload(Context)
reload(plant)
reload(plant_so)


if scale==21:
    df_DoE=df4
    df_data=df2

elif scale==25:
    df_DoE=df5
    df_data=df3
else:
    print 'Wrong Scale'
    () + 1
status=df_DoE['Finished'].tolist()[index]
#if status == 'Completed':
#    continue
iFlow=df_DoE['Flow Speed'].tolist()[index]
iDepth=df_DoE['Depth'].tolist()[index]
iOffset=df_DoE['Offset'].tolist()[index]
tagFlow=df_data['Speed Tags'].tolist()[iFlow]
tagDepth=df_data['Depth Tags'].tolist()[iDepth]
tagOffset=df_data['Offset Tags'].tolist()[iOffset]

flowSpeed=df_data['Flow Speed'].tolist()[iFlow]
waterLevel=df_data['Depth'].tolist()[iDepth]
offset=df_data['Offset'].tolist()[iOffset]

#Change Context
plant.opts.water_level=waterLevel
plant.opts.inflow_velocity=flowSpeed
plant.opts.outflow_velocity=flowSpeed
plant.opts.caisson_scale=float(scale)
plant.opts.caisson_Yoffset=-offset


plant.opts.dt_fixed = dt
plant.opts.T = T
plant.opts.he = Tank_he
plant.opts.he_caisson=Caisson_he

plant.Update_Model()

Context.setFromModule(plant,mutable=True)
ct = Context.get()
# numerical simulation options
plant_so.ct=ct
so = plant_so
so.tnList = [0.0,plant.dt_init]+[i*plant.dt_fixed for i in range(1,plant.nDTout+1)]
info = open("TimeList.txt","w")
for time in so.tnList:
    info.write(str(time)+"\n")
info.close()
plant_so.tnList=so.tnList
pList=[]
nList=[]
so.sList=[]
OptDB = PETSc.Options()
for (p,n) in so.pnList:
    so.sList.append(default_s)
    pList.append(__import__(p))
    reload(pList[-1])
    nList.append(__import__(n))
    reload(nList[-1])
    pList[-1].name = p
    nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
    nList[-1].levelLinearSolver = default_n.KSP_petsc4py
    OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
opts.save_dof = True
opts.dataDir='.'
opts.probDir='.'
opts.logLevel=7
opts.verbose=True
opts.viewMesh=True

In [224]:
%%px 
ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)

In [225]:
%%px --noblock
from threading import Thread
simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))

<AsyncResult: execute>

In [226]:
%%px 
simulation_thread.start()

In [227]:
#view['stop'] = True

In [ ]:
import numpy as np
ranks = view['rank']
rank_indices = np.argsort(ranks)

def mpi_order(seq):
    """Return elements of a sequence ordered by MPI rank.

    The input sequence is assumed to be ordered by engine ID."""
    return [seq[x] for x in rank_indices]

showTriangles=True
showAir=False
def plot_current_results():
    global t_0, plt0
    """Makes a blocking call to retrieve remote data and displays the solution mesh
    as a contour plot.
    
    Parameters
    ----------
    in_place : bool
        By default it calls clear_output so that new plots replace old ones.  Set
        to False to allow keeping of all previous outputs.
    """
    import numpy as np
    import matplotlib.tri as mtri
    import math
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    view.apply_sync(load_simulation_globals)
    x = np.concatenate(mpi_order(view['x']))
    y = np.concatenate(mpi_order(view['y']))
    u = np.concatenate(mpi_order(view['u']))
    v = np.concatenate(mpi_order(view['v']))
    vof = np.concatenate(mpi_order(view['vof']))
    shifts = np.cumsum([0]+mpi_order(view['nn'])[:-1])
    flat_triangles = np.concatenate([ tri + shift for tri,shift in zip(mpi_order(view['triangles']), shifts) ])

    t = mpi_order(view['t'])[0]
    phi = np.concatenate(mpi_order(view['phi']))
    cfl = mpi_order(view['cfl'])[0]
    p = np.concatenate(mpi_order(view['p']))
    dt = mpi_order(view['dt'])[0]

    Vmax=np.amax(np.sqrt(u[:]**2 + v[:]**2))
    print "u_max={0:.3f}, v_max={1:.3f}, Vmax={2:.3f}, cfl={3:.3f}, dt={4:.5f}".format(np.amax(u[:]),
                                                           np.amax(v[:]),
                                                           Vmax,
                                                           np.asarray(cfl).max()*dt,
                                                           dt
                                                           )
    #if math.isnan(np.amax(u[:])):
    #    return plt0
    f.clear()
    triang = mtri.Triangulation(x, y, flat_triangles)

    dim=[x.max(),y.max()]
    xg = np.linspace(0, dim[0], 20)
    yg = np.linspace(0, dim[1], 20)
    xi, yi = np.meshgrid(xg,yg)
    
    wvof = np.ones(vof.shape,'d')
    if showAir:
        wvof = vof
    else:
        wvof -= vof
            
    plt.xlabel(r'z[m]')
    plt.ylabel(r'x[m]')
   
    #domain = plant.domain
    #def get_N_RGBCol(N=5):
    #    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    #    return HSV_tuples
    #colors=get_N_RGBCol(2*len(domain.segments))
    #for si,s in enumerate(domain.segments):
    #        plt.plot([domain.vertices[s[0]][0],
    #                     domain.vertices[s[1]][0]],
    #                     [domain.vertices[s[0]][1],
    #                     domain.vertices[s[1]][1]],
    #                     color=colors[domain.segmentFlags[si]-1],
    #                     linewidth=2,
    #                     marker='o')



    if showTriangles==True:
        plt.triplot(x,y,flat_triangles, linewidth=0.5)
    plt.tricontourf(x,y,flat_triangles,np.sqrt(u[:]**2 + v[:]**2))
    clb=plt.colorbar(orientation='horizontal')
    #clb = plt.colorbar()
    #clb.set_label('Vmax', labelpad=-40, y=1.05, rotation=0)
    clb.set_label('Vmax')
    #plt.tricontourf(x,y,triangles,wvof)
    plt.tricontour(x,y,flat_triangles,phi,[0], linewidth=4)
    #u_interp_lin = mtri.LinearTriInterpolator(triang, u[:])
    #v_interp_lin = mtri.LinearTriInterpolator(triang, v[:])
    #u_lin = u_interp_lin(xi, yi)
    #v_lin = v_interp_lin(xi, yi)
    #plt.streamplot(xg, yg, u_lin, v_lin,color='k')
    plt.title('T=%3.5e' % (t,))
    plt.xlim((x.min(),dim[0]))
    t_0=t
    f.canvas.draw()
    plt0=plt
    return plt

'''Some plotting and monitoring utilities
'''
def load_simulation_globals():
    """Put some variables we need in engine namespace.

    These can then be retrieved by clients for inspection, visualization, etc.
    """
    global nn, x, y, vof, triangles, t, phi, u, v, cfl, p, dt
    model_vof = ns.modelList[1].levelModelList[-1]
    model_ls = ns.modelList[2].levelModelList[-1]
    # save solution and grid data for plotting purposes
    cfl = ns.modelList[0].levelModelList[-1].q[('cfl',0)]
    dt = ns.systemStepController.dt_system
    x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,0]
    y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,1]
    triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray
    p = ns.modelList[0].levelModelList[-1].u[0].dof
    u = ns.modelList[0].levelModelList[-1].u[1].dof
    v = ns.modelList[0].levelModelList[-1].u[2].dof
    vof = ns.modelList[1].levelModelList[-1].u[0].dof_last
    phi = ns.modelList[2].levelModelList[-1].u[0].dof_last
    nn = len(x)
    #print "p={0}, u={1}, v={2}, triangles={3}, vof={4}, phi={5}".format(len(p),len(u),len(v),len(triangles),len(vof),len(phi))
    t=ns.systemStepController.t_system
    cfl*=dt
    
def simulation_alive():
    """Return True if the simulation thread is still running on any engine.
    """
    return any(view.apply_sync(lambda: simulation_thread.is_alive()))

def monitor_simulation(refresh=5.0):
    global t_0
    """Monitor the simulation progress and call plotting routine.

    Supress KeyboardInterrupt exception if interrupted, ensure that the last 
    figure is always displayed and provide basic timing and simulation status.

    Parameters
    ----------
    refresh : float
      Refresh interval between calls to retrieve and plot data.  The default
      is 5s, adjust depending on the desired refresh rate, but be aware that 
      very short intervals will start having a significant impact.

    """
    import datetime as dt, time
    if not simulation_alive():
        f.clear()
        plt=plot_current_results()
        f.canvas.draw()
        print 'Simulation has already finished, no monitoring to do.'
        error= True
        return error
    t_0=0.
    t0 = dt.datetime.now()
    try:
        while simulation_alive():

            plt=plot_current_results()
            error= False
            tmon = dt.datetime.now() - t0
            print 'Monitored for: %s. at t=%12.5e' % (tmon,view.apply_sync(lambda: ns.systemStepController.t_system)[0])
            time.sleep(refresh) # so we don't hammer the server too fast
    except (KeyboardInterrupt):#, error.TimeoutError):
        msg = 'Monitoring interrupted, simulation is ongoing!'
    else:
        if view.apply_sync(lambda: ns.systemStepController.converged()):
            msg =  "\x1b[31mStep Failed at t=%12.5e \x1b[0m" % (view.apply_sync(lambda: ns.systemStepController.t_system)[0])
            error= True
        else:
            msg = 'Simulation completed!'

    tmon = dt.datetime.now() - t0
    print msg
    print 'Monitored for: %s.' % tmon
    return error
    
    


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
f=plt.figure()
plt.plot([])
f.canvas.draw()
#parUtilities.view=view
#parUtilities.plt=plt
#parUtilities.f=plt.figure()
#parUtilities.showTriangles=True
#parUtilities.showAir=False
#error=parUtilities.monitor_simulation(refresh=5.0)

showTriangles=True
showAir=False
monitor_simulation(refresh=5.0)

u_max=0.311, v_max=0.340, Vmax=0.341, cfl=21.974, dt=0.00020
Monitored for: 0:00:06.524172. at t= 3.00000e-03
u_max=0.324, v_max=0.368, Vmax=0.368, cfl=22.342, dt=0.00100
Monitored for: 0:00:17.979401. at t= 1.60000e-02
u_max=0.332, v_max=0.349, Vmax=0.350, cfl=23.001, dt=0.00200
Monitored for: 0:00:30.303276. at t= 2.90000e-02
u_max=0.353, v_max=0.347, Vmax=0.353, cfl=23.538, dt=0.00100
Monitored for: 0:00:42.227185. at t= 4.30000e-02
u_max=0.364, v_max=0.345, Vmax=0.364, cfl=24.001, dt=0.00200
Monitored for: 0:00:54.344465. at t= 5.50000e-02
u_max=0.357, v_max=0.342, Vmax=0.358, cfl=24.406, dt=0.00100
Monitored for: 0:01:06.359525. at t= 6.90000e-02
u_max=0.378, v_max=0.338, Vmax=0.378, cfl=24.782, dt=0.00200
Monitored for: 0:01:18.383428. at t= 8.20000e-02
u_max=0.389, v_max=0.333, Vmax=0.389, cfl=25.080, dt=0.00100
Monitored for: 0:01:30.893195. at t= 9.50000e-02
u_max=0.399, v_max=0.327, Vmax=0.399, cfl=25.334, dt=0.00100
Monitored for: 0:01:42.488270. at t= 1.07000e-01
u_max=0.40

In [ ]:
import os   
checkpath=os.path.exists('./Results') 
if not checkpath:
        os.mkdir("Results")
ExpTag='T_' + str(index) + "_S_"+ str(scale)
if not error:
    !rm ./Results/{ExpTag}_forceHistory_p.txt
    !rm ./Results/{ExpTag}_forceHistory_v.txt
    !cp forceHistory_p.txt ./Results/{ExpTag}_forceHistory_p.txt
    !cp forceHistory_v.txt  ./Results/{ExpTag}_forceHistory_v.txt
    Exp_Status='Completed'
else:
    Exp_Status='Crashed'
#!rm ./Results/{ExpTag}_p.xmf
#!rm ./Results/{ExpTag}_p.h5
#!cp tempFlume_p.xmf ./Results/{ExpTag}_p.xmf
#!cp tempFlume_p.h5  ./Results/{ExpTag}_p.h5
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('DoE_TempFlume.xlsx', engine='xlsxwriter')

if scale==21:
    df4.loc[index,'Finished']=Exp_Status

elif scale==25:
    df5.loc[index,'Finished']=Exp_Status
else:
    print 'Scale Error'

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer, sheet_name='Parameters')
df2.to_excel(writer, sheet_name='Parameters Scale 1-21')
df3.to_excel(writer, sheet_name='Parameters Scale 1-25')
df4.to_excel(writer, sheet_name='Tests Scale 1-21')
df5.to_excel(writer, sheet_name='Tests Scale 1-25')
# Close the Pandas Excel writer and output the Excel file.
writer.save()